In [1]:
from qiskit import *
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-utokyo',group='internal',project='icepp')
#provider.backends(simulator=False)

In [2]:
backend = provider.get_backend('ibmq_jakarta')

In [3]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits, GatesetTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

# Auxiliary methods
from qiskit.quantum_info import Choi, Kraus
from qiskit.extensions import HGate, XGate

In [4]:
# Process tomography of a single-qubit gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

/opt/anaconda3/lib/python3.8/site-packages/qiskit/ignis/verification/tomography/basis/circuits.py:465: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  prep += preparation(prep_label[j], prep_qubits[j])
/opt/anaconda3/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:942: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)
/opt/anaconda3/lib/python3.8/site-packages/qiskit/ignis/verification/tomography/basis/circuits.py:478: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  circ = prep + meas
/opt/

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'1': 1989, '0': 2011},
 (('Zp',), ('Z',)): {'1': 1994, '0': 2006},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'0': 1985, '1': 2015},
 (('Zm',), ('Z',)): {'0': 1984, '1': 2016},
 (('Xp',), ('X',)): {'1': 2028, '0': 1972},
 (('Xp',), ('Y',)): {'0': 1993, '1': 2007},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'1': 2028, '0': 1972},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'0': 1958, '1': 2042}}

In [5]:
# Get the ideal unitary operator
target_unitary = qi.Operator(circ)
print(target_unitary)

# Tomographic reconstruction
choi_fit_lstsq = qpt_tomo.fit(method='lstsq')
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitary)))

Operator([[ 0.70710678+0.j,  0.70710678+0.j],
          [ 0.70710678+0.j, -0.70710678+0.j]],
         input_dims=(2,), output_dims=(2,))
Average gate fidelity: F = 0.99840


In [6]:
# Process tomography of a single-qubit gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, backend, shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 3801, '1': 199},
 (('Zp',), ('Y',)): {'0': 1944, '1': 2056},
 (('Zp',), ('Z',)): {'0': 1996, '1': 2004},
 (('Zm',), ('X',)): {'0': 271, '1': 3729},
 (('Zm',), ('Y',)): {'0': 2174, '1': 1826},
 (('Zm',), ('Z',)): {'0': 1987, '1': 2013},
 (('Xp',), ('X',)): {'0': 1940, '1': 2060},
 (('Xp',), ('Y',)): {'0': 2104, '1': 1896},
 (('Xp',), ('Z',)): {'0': 3798, '1': 202},
 (('Yp',), ('X',)): {'0': 2017, '1': 1983},
 (('Yp',), ('Y',)): {'0': 216, '1': 3784},
 (('Yp',), ('Z',)): {'0': 2123, '1': 1877}}

In [7]:
# Get the ideal unitary operator
target_unitary = qi.Operator(circ)
print(target_unitary)

# Tomographic reconstruction
choi_fit_lstsq = qpt_tomo.fit(method='lstsq')
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitary)))

Operator([[ 0.70710678+0.j,  0.70710678+0.j],
          [ 0.70710678+0.j, -0.70710678+0.j]],
         input_dims=(2,), output_dims=(2,))
Average gate fidelity: F = 0.95063


In [9]:
# Process tomography of a two-qubit gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.cx(q[0], q[1])

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qubit_list = [q[0], q[1]]
qpt_circs = process_tomography_circuits(circ, qubit_list, prepared_qubits=qubit_list)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp', 'Zp'), ('X', 'X')): {'01': 509, '10': 530, '00': 490, '11': 471},
 (('Zp', 'Zp'), ('X', 'Y')): {'01': 491, '11': 534, '10': 481, '00': 494},
 (('Zp', 'Zp'), ('X', 'Z')): {'00': 1014, '01': 986},
 (('Zp', 'Zp'), ('Y', 'X')): {'11': 500, '00': 479, '10': 502, '01': 519},
 (('Zp', 'Zp'), ('Y', 'Y')): {'01': 497, '00': 490, '11': 527, '10': 486},
 (('Zp', 'Zp'), ('Y', 'Z')): {'01': 980, '00': 1020},
 (('Zp', 'Zp'), ('Z', 'X')): {'10': 990, '00': 1010},
 (('Zp', 'Zp'), ('Z', 'Y')): {'10': 1019, '00': 981},
 (('Zp', 'Zp'), ('Z', 'Z')): {'00': 2000},
 (('Zp', 'Zm'), ('X', 'X')): {'01': 495, '11': 533, '10': 491, '00': 481},
 (('Zp', 'Zm'), ('X', 'Y')): {'01': 497, '10': 506, '11': 483, '00': 514},
 (('Zp', 'Zm'), ('X', 'Z')): {'10': 1001, '11': 999},
 (('Zp', 'Zm'), ('Y', 'X')): {'01': 481, '10': 524, '00': 500, '11': 495},
 (('Zp', 'Zm'), ('Y', 'Y')): {'11': 487, '00': 513, '10': 508, '01': 492},
 (('Zp', 'Zm'), ('Y', 'Z')): {'10': 1013, '11': 987},
 (('Zp', 'Zm'), ('Z', 'X')): {'10

In [10]:
# Get the ideal unitary operator
target_unitary = qi.Operator(circ)
print(target_unitary)

# Tomographic reconstruction
choi_fit = qpt_tomo.fit(method='lstsq')
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit, target=target_unitary)))

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Average gate fidelity: F = 0.98940


In [11]:
# Process tomography of a two-qubit gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.cx(q[0], q[1])

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qubit_list = [q[0], q[1]]
qpt_circs = process_tomography_circuits(circ, qubit_list, prepared_qubits=qubit_list)
job = qiskit.execute(qpt_circs, backend, shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp', 'Zp'), ('X', 'X')): {'00': 574, '01': 512, '10': 488, '11': 426},
 (('Zp', 'Zp'), ('X', 'Y')): {'00': 497, '01': 438, '10': 562, '11': 503},
 (('Zp', 'Zp'), ('X', 'Z')): {'00': 1063, '01': 907, '10': 10, '11': 20},
 (('Zp', 'Zp'), ('Y', 'X')): {'00': 551, '01': 482, '10': 479, '11': 488},
 (('Zp', 'Zp'), ('Y', 'Y')): {'00': 495, '01': 465, '10': 569, '11': 471},
 (('Zp', 'Zp'), ('Y', 'Z')): {'00': 1057, '01': 913, '10': 15, '11': 15},
 (('Zp', 'Zp'), ('Z', 'X')): {'00': 998, '01': 9, '10': 972, '11': 21},
 (('Zp', 'Zp'), ('Z', 'Y')): {'00': 931, '01': 12, '10': 1040, '11': 17},
 (('Zp', 'Zp'), ('Z', 'Z')): {'00': 1940, '01': 30, '10': 18, '11': 12},
 (('Zp', 'Zm'), ('X', 'X')): {'00': 486, '01': 468, '10': 558, '11': 488},
 (('Zp', 'Zm'), ('X', 'Y')): {'00': 583, '01': 556, '10': 457, '11': 404},
 (('Zp', 'Zm'), ('X', 'Z')): {'00': 37, '01': 30, '10': 1062, '11': 871},
 (('Zp', 'Zm'), ('Y', 'X')): {'00': 541, '01': 461, '10': 518, '11': 480},
 (('Zp', 'Zm'), ('Y', 'Y')): {'00'

In [12]:
# Get the ideal unitary operator
target_unitary = qi.Operator(circ)
print(target_unitary)

# Tomographic reconstruction
choi_fit = qpt_tomo.fit(method='lstsq')
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit, target=target_unitary)))

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Average gate fidelity: F = 0.92117


In [13]:
# Process tomography of a three-qubit gate
q = QuantumRegister(3)
circ = QuantumCircuit(q)
circ.ccx(q[0], q[1], q[2])

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qubit_list = [q[0], q[1], q[2]]
qpt_circs = process_tomography_circuits(circ, qubit_list, prepared_qubits=qubit_list)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=500)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp', 'Zp', 'Zp'), ('X', 'X', 'X')): {'110': 64,
  '100': 68,
  '001': 62,
  '011': 72,
  '111': 67,
  '000': 59,
  '010': 59,
  '101': 49},
 (('Zp', 'Zp', 'Zp'), ('X', 'X', 'Y')): {'010': 61,
  '000': 61,
  '100': 59,
  '110': 63,
  '001': 50,
  '011': 63,
  '111': 79,
  '101': 64},
 (('Zp', 'Zp', 'Zp'), ('X', 'X', 'Z')): {'001': 116,
  '010': 129,
  '000': 138,
  '011': 117},
 (('Zp', 'Zp', 'Zp'), ('X', 'Y', 'X')): {'110': 72,
  '100': 59,
  '001': 67,
  '111': 57,
  '011': 58,
  '101': 53,
  '000': 55,
  '010': 79},
 (('Zp', 'Zp', 'Zp'), ('X', 'Y', 'Y')): {'101': 59,
  '010': 69,
  '000': 50,
  '111': 64,
  '011': 60,
  '001': 54,
  '110': 74,
  '100': 70},
 (('Zp', 'Zp', 'Zp'), ('X', 'Y', 'Z')): {'001': 99,
  '000': 149,
  '010': 127,
  '011': 125},
 (('Zp', 'Zp', 'Zp'), ('X', 'Z', 'X')): {'100': 106,
  '000': 130,
  '101': 117,
  '001': 147},
 (('Zp', 'Zp', 'Zp'), ('X', 'Z', 'Y')): {'001': 149,
  '100': 111,
  '000': 121,
  '101': 119},
 (('Zp', 'Zp', 'Zp'), ('X', 'Z', 'Z')): {

In [14]:
# Get the ideal unitary operator
target_unitary = qi.Operator(circ)
print(target_unitary)

# Tomographic reconstruction
choi_fit = qpt_tomo.fit(method='lstsq')
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit, target=target_unitary)))

Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2, 2), output_dims=(2, 2, 2))
Average gate fidelity: F = 0.96090
